In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import math
from py_files.d2v_embed import d2v_embed

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [2]:
df = pd.read_csv('df_with_features.csv', index_col=0)

C:\Users\Roan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (17950,17951,17953,17954,17955) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,primaryTitleFormatted,Year,oscar_noms,...,nm9985837,budget,genres_y,original_language,popularity,production_companies,tagline,Keywords,revenue,overview
tt0010600,The Doll,Die Puppe,1919.0,NaN,66.0,1898.0,True,the_doll,1919,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0011841,Way Down East,Way Down East,1920.0,NaN,145.0,5376.0,True,way_down_east,1920,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0012494,Déstiny,Der müde Tod,1921.0,NaN,97.0,5842.0,True,destiny,1921,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0015163,The Navigator,The Navigator,1924.0,NaN,59.0,9652.0,True,the_navigator,1924,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925.0,NaN,93.0,17887.0,True,the_phantom_of_the_opera,1925,3.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7959 entries, tt0010600 to tt9911196
Columns: 17957 entries, primaryTitle to overview
dtypes: bool(3), float64(33), int64(17909), object(12)
memory usage: 1.1+ GB


In [5]:
df.select_dtypes(include=['bool']).columns

Index(['label', 'hasRemake', 'isEN'], dtype='object')

In [6]:
df.select_dtypes(include=['float']).columns

Index(['startYear', 'endYear', 'runtimeMinutes', 'numVotes', 'oscar_noms',
       'oscar_wins', 'Rank', 'Worldwide', 'Domestic', 'Foreign', 'year',
       'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'budget', 'popularity', 'revenue'],
      dtype='object')

In [7]:
df.select_dtypes(include=['object']).columns

Index(['primaryTitle', 'originalTitle', 'primaryTitleFormatted',
       'title_language', 'titleFormatted', 'genres_x', 'genres_y',
       'original_language', 'production_companies', 'tagline', 'Keywords',
       'overview'],
      dtype='object')

In [8]:
prim_title_df = d2v_embed(df['primaryTitle'])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:48<00:00,  2.28s/it]


In [9]:
orig_title_df = d2v_embed(df['originalTitle'])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:58<00:00,  2.39s/it]


In [10]:
prim_title_formatted_df = d2v_embed(df['primaryTitleFormatted'])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:31<00:00,  1.51s/it]


In [11]:
title_formatted_df = d2v_embed(df['titleFormatted'])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:27<00:00,  1.15it/s]


In [12]:
genres_df = d2v_embed(df['genres'])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:30<00:00,  1.11it/s]


In [13]:
df['title_language'] = pd.factorize(df['title_language'])[0]

In [14]:
df.drop(columns = df.select_dtypes(include='object').columns, inplace=True)

In [15]:
df.head()

,startYear,endYear,runtimeMinutes,numVotes,label,Year,oscar_noms,oscar_wins,Rank,Worldwide,...,nm9925241,nm9933959,nm9942830,nm9946633,nm9955258,nm9958352,nm9958353,nm9980769,nm9985316,nm9985837
tt0010600,1919.0,NaN,66.0,1898.0,True,1919,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0011841,1920.0,NaN,145.0,5376.0,True,1920,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0012494,1921.0,NaN,97.0,5842.0,True,1921,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0015163,1924.0,NaN,59.0,9652.0,True,1924,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0016220,1925.0,NaN,93.0,17887.0,True,1925,3.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# dealing with nan values
for index, row in df.iterrows():
    # For missing startYear or endYear entries, insert the other, if it exists.
    if math.isnan(row['startYear']):
        if not math.isnan(row['endYear']):
            df.at[index,'startYear']=df.at[index,'endYear']
    if math.isnan(row['endYear']):
        if not math.isnan(row['startYear']):
            df.at[index,'endYear']=df.at[index,'startYear']
    
    # For missing oscar_noms and oscar_wins, insert 0
    if math.isnan(row['oscar_noms']):
        df.at[index,'oscar_noms'] = 0
    if math.isnan(row['oscar_wins']):
        df.at[index,'oscar_wins'] = 0

        
df.head()

,startYear,endYear,runtimeMinutes,numVotes,label,Year,oscar_noms,oscar_wins,Rank,Worldwide,...,nm9925241,nm9933959,nm9942830,nm9946633,nm9955258,nm9958352,nm9958353,nm9980769,nm9985316,nm9985837
tt0010600,1919.0,1919.0,66.0,1898.0,True,1919,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0011841,1920.0,1920.0,145.0,5376.0,True,1920,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0012494,1921.0,1921.0,97.0,5842.0,True,1921,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0015163,1924.0,1924.0,59.0,9652.0,True,1924,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0016220,1925.0,1925.0,93.0,17887.0,True,1925,3.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df['numVotes'] = df['numVotes'].fillna(df['numVotes'].mean(skipna=True))
df['runtimeMinutes'] = df['runtimeMinutes'].fillna(df['runtimeMinutes'].mean(skipna=True))

df.head()

,startYear,endYear,runtimeMinutes,numVotes,label,Year,oscar_noms,oscar_wins,Rank,Worldwide,...,nm9925241,nm9933959,nm9942830,nm9946633,nm9955258,nm9958352,nm9958353,nm9980769,nm9985316,nm9985837
tt0010600,1919.0,1919.0,66.0,1898.0,True,1919,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0011841,1920.0,1920.0,145.0,5376.0,True,1920,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0012494,1921.0,1921.0,97.0,5842.0,True,1921,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0015163,1924.0,1924.0,59.0,9652.0,True,1924,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
tt0016220,1925.0,1925.0,93.0,17887.0,True,1925,3.0,0.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df = df.join(prim_title_df)
df = df.join(orig_title_df)
df = df.join(prim_title_formatted_df)
df = df.join(title_formatted_df)
df = df.join(genres_df)
df.head()

,startYear,endYear,runtimeMinutes,numVotes,label,Year,oscar_noms,oscar_wins,Rank,Worldwide,...,genres_118,genres_119,genres_120,genres_121,genres_122,genres_123,genres_124,genres_125,genres_126,genres_127
tt0010600,1919.0,1919.0,66.0,1898.0,True,1919,0.0,0.0,NaN,NaN,...,0.179899,0.132897,0.052456,-0.220462,-0.088191,-0.075018,0.104221,0.043458,-0.011547,0.047500
tt0011841,1920.0,1920.0,145.0,5376.0,True,1920,0.0,0.0,NaN,NaN,...,-0.038521,-0.031211,-0.010481,0.047988,0.006431,-0.095900,-0.089550,-0.068009,0.025714,-0.183766
tt0012494,1921.0,1921.0,97.0,5842.0,True,1921,0.0,0.0,NaN,NaN,...,-0.001869,-0.003446,-0.002707,0.001183,0.002295,0.002347,0.001741,0.000817,0.000066,0.001884
tt0015163,1924.0,1924.0,59.0,9652.0,True,1924,0.0,0.0,NaN,NaN,...,0.000613,-0.003880,-0.001888,0.001791,0.003596,0.002096,-0.001731,-0.002269,0.000080,-0.000290
tt0016220,1925.0,1925.0,93.0,17887.0,True,1925,3.0,0.0,NaN,NaN,...,-0.003687,0.000546,-0.000911,-0.003678,0.000535,0.002635,-0.002205,0.001363,-0.000875,-0.001600


In [19]:
df.columns[df.isna().any()].tolist()

['Rank',
 'Worldwide',
 'Domestic',
 'Foreign',
 'year',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [20]:
df.to_csv('df_with_features_fully_processed.csv')

In [21]:
# split in train and validation sets
train_df, valid_df = train_test_split(df, train_size=0.7, shuffle=True, stratify=df['label'], random_state=17)

In [22]:
model_lgbm = lgb.LGBMClassifier(objective='binary',
                                learning_rate=0.01,
                                num_iterations=1000,
                                feature_fraction=0.8,
                                verbosity=1,
                                random_state=17)
model_lgbm.fit(train_df.loc[:, train_df.columns != 'label'],
              train_df['label'],
              eval_metric='logloss')

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] Number of positive: 2793, number of negative: 2778
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 165002
[LightGBM] [Info] Number of data points in the train set: 5571, number of used features: 674
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501346 -> initscore=0.005385
[LightGBM] [Info] Start training from score 0.005385


LGBMClassifier(feature_fraction=0.8, learning_rate=0.01, num_iterations=1000,
               objective='binary', random_state=17, verbosity=1)

In [23]:
valid_df['label'] = valid_df['label'].astype('int')
val_preds = model_lgbm.predict(valid_df.loc[:, train_df.columns != 'label'])
acc_valid_lgbm = accuracy_score(y_true=valid_df['label'].astype('int'), y_pred=val_preds)
acc_valid_lgbm

0.7839195979899497

In [45]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier()
model_xgb.fit(train_df.loc[:, train_df.columns != 'label'],
              train_df['label'])

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:01:48] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [47]:
valid_df['label'] = valid_df['label'].astype('int')
val_preds = model_xgb.predict(valid_df.loc[:, train_df.columns != 'label'])
acc_valid_xgb = accuracy_score(y_true=valid_df['label'].astype('int'), y_pred=val_preds)
acc_valid_xgb

0.7772194304857621